In [ ]:
## to create virtual env for jupyter notebook
# conda create --no-default-packages -n waste python=3.6 pandas numpy scipy scikit-learn matplotlib
# conda install --name waste --copy -c conda-forge folium
# source activate waste
# # sankey charts work in jupyter notebook but not in jupyter lab
# pip install floweaver
# pip install jupyterlab
# pip install ipysankeywidget
# jupyter nbextension enable --py --sys-prefix ipysankeywidget
# pip install --user floweaver ipysankeywidget
# jupyter nbextension enable --py widgetsnbextension --sys-prefix
# python -m ipykernel install --user --name waste --display-name "UK waste flows 3.6"

 # Get data
 
 http://www.wastedataflow.org/reports/default.aspx

In [1]:
import pandas as pd
import numpy as np
import json
import folium
import floweaver

# pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [2]:
df1 = pd.read_excel('./datasets/Raw_Data_Download_London_quarter.xlsx', sheet_name='Q100', skiprows=0, header=1)
df2 = pd.read_excel('./datasets/2019/Raw Data Download.xlsx', sheet_name='Q100', skiprows=0, header=1)
df = pd.concat([df1, df2], ignore_index=True)

# Data Cleaning and Definitions

In [3]:
df['year'] = df['Period'].str.split(' ').str[1]
df['MaterialGroup'] = df['MaterialGroup'].replace(np.nan,'Unknown',regex=True).replace('','Unknown',regex=True)
df['FacilityType'] = df['FacilityType'].replace(np.nan,'Unknown',regex=True).replace('','Unknown',regex=True)
df['Material'] = df['Material'].replace(np.nan,'Unknown',regex=True).replace('','Unknown',regex=True)
df['WasteStreamType'] = df['WasteStreamType'].replace(np.nan,'Unknown',regex=True).replace('','Unknown',regex=True)
df['OutputProcessType'] = df['OutputProcessType'].replace(np.nan,'Unknown',regex=True).replace('','Unknown',regex=True)

df['FacilityPostCode'] = df['FacilityPostCode'].replace(np.nan,'',regex=True)
df['FacilityName'] = df['FacilityName'].replace(np.nan,'',regex=True)
df['FacilityAddress'] = df['FacilityAddress'].replace(np.nan,'',regex=True)

# manually add in Waste Management Authority for London Boroughs which share waste management services
wma = {'North London Waste Authority': 
           ['North London Waste Authority', 'Barnet LB', 'Camden LB', 'Enfield LB', 'Hackney LB', 'Haringey LB', 'Islington LB', 'Waltham Forest LB'], 
       'Western Riverside Waste Authority': 
           ['Western Riverside Waste Authority', 'Hammersmith and Fulham LB', 'Royal Borough of Kensington and Chelsea', 'Lambeth LB', 'Wandsworth LB'], 
       'West London Waste Authority': 
           ['West London Waste Authority', 'Brent LB', 'Ealing LB', 'Harrow LB', 'Hillingdon LB', 'Hounslow LB', 'Richmond upon Thames LB'], 
       'South London Waste Partnership': 
           ['South London Waste Partnership', 'Croydon LB', 'Royal Borough of Kingston upon Thames ', 'Merton LB', 'Sutton LB'], 
       'East London Waste Authority': 
           ['East London Waste Authority', 'Barking and Dagenham LB', 'Havering LB', 'Newham LB', 'Redbridge LB']
      }
    
    
wma_reverse = {}
for k,v in wma.items():
    for vl in v:
        if vl not in wma_reverse.keys():
            wma_reverse[vl] = k
        else:
            wma_reverse[vl].add(set(k))

df['WasteManagementAuthority'] = df[['Authority']].apply(lambda x: wma_reverse[x[0]] if x[0] in wma_reverse.keys() else x[0], axis =1)

def get_quarter(year, period):
    m = period.split(' ')[0]
    if m == 'Jan':
        return int(year + '1')
    elif m == 'Apr':
        return int(year + '2')
    elif m == 'Jul':
        return int(year + '3')
    elif m == 'Oct':
        return int(year + '4')
    else:
        return int(year + '5')
    
# Create integer quarter - so that could filter flows on this property (quarter greater than)                 
df['Quarter'] = df[['Period', 'year']].astype(str).apply(lambda row: get_quarter(row[1], row[0]), axis = 1)

# TotalTonnes matches the sum of TonnesByMaterial across materials within same processing facility,
# with the same figure repeated across several material lines
# we use TonnesBy Material when material is known and TotalTonnes when Material is Unknown or it's a Process Loss
df['Tonnes'] = df[['Material', 'TotalTonnes', 'TonnesByMaterial']].apply(lambda row: row[1] if row[0] in ["Unknown"] else row[2], axis = 1)

# unique id of row for duplication checks and defining flow id
#df['rownum'] = pd.Series(np.arange(len(df))).values

# Geocode facility locations 

There are no unique ids for facilities, facility names and addresses are manually entered with many errors

Geocoding them with google maps api to get unique id and location coordinates 

In [4]:
# create a list of facility names and addresses to create a lookup table
# facility_addresses = df[(df['FacilityAddress']!='') | (df['FacilityName']!='') | (df['FacilityPostCode']!='')][
#     ['NationalFacilityId', 'FacilityName','FacilityAddress', 'FacilityPostCode']
#     ].groupby(['FacilityName','FacilityAddress', 'FacilityPostCode']).count().reset_index()


In [5]:
# geo_location = {}

# for loc in locations:
#     params = {
#     'address': loc,
#     'key' : api_key    
#     }
#     req = requests.get(GOOGLE_MAPS_API_URL, params=params)
#     res = req.json()
    
#     if res['status'] == 'ZERO_RESULTS':
#         geodata = []
#         # put unknown into atlantic ocean
#         geodata.append(14.5994)
#         geodata.append(28.6731)
#         geodata.append('location unknown')
#         geodata.append('unknown')
 
#     else:    
#         result = res['results'][0]
#         geodata = []
#         geodata.append(result['geometry']['location']['lat'])
#         geodata.append(result['geometry']['location']['lng'])
#         geodata.append(result['formatted_address'])
#         geodata.append(result['place_id'])

    
#     geo_location[loc] = geodata
    

In [6]:
# with open('./datasets/geocoded.json', 'w') as fp:
#     json.dump(geo_location, fp)

In [7]:
with open('./datasets/geocoded.json', 'r') as f:
    geo_location = json.load(f)

In [8]:
df['address'] = df[['FacilityName','FacilityAddress', 'FacilityPostCode']].apply(lambda x: 
                                                         str(x[1]) + ', ' + str(x[2]) if x[0] == 'Other/Exempt' 
                                                        else str(x[0]) + ', ' + str(x[1]) + ', ' + str(x[2])
                                                         , axis =1)

df['Location_Coordinates'] = df['address'].apply(lambda x: [geo_location[x][0],geo_location[x][1]] if x != ', , ' else [14.5994,28.6731])
df['FacilityAddressGeo'] = df['address'].apply(lambda x: geo_location[x][2] if x != ', , ' else '')
df['FacilityIdGeo'] = df['address'].apply(lambda x: geo_location[x][3] if x != ', , ' else '')
df['FacilityIdGenerated'] = df[['NationalFacilityId','FacilityTypeId','FacilityIdGeo']].apply(lambda x: str(x[0]) + "_" + str(x[1]) + "_" + str(x[2]), axis = 1)

# Define graph edges

In [10]:
# Initial houshold waste streams at source (collected by Local Authority)
df_stream_same = df[ ((df['WasteProcessorOutputId']==0)&(df['SenderWasteProcessorOutputId']==0))  
                    & ((df['MaterialId']!=0)
                        | ( (df['FacilityType'] =='Advanced thermal treatment') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Anaerobic or Aerobic Digestion Segregated') & (df['WasteStreamType'] == 'Food waste')) 
                        | ( (df['FacilityType'] =='Anaerobic or Aerobic Digestion Segregated') & (df['WasteStreamType'] == 'Green waste')) 
                        | ( (df['FacilityType'] =='Anaerobic or Aerobic Digestion Segregated') & (df['WasteStreamType'] == 'Mixed green and food waste')) 
                        | ( (df['FacilityType'] =='Exporter - recycling (qu19)') & (df['WasteStreamType'] == 'Source segregated recyclate')) 
                        | ( (df['FacilityType'] =='Exporter - reuse (qu35)') & (df['WasteStreamType'] == 'Source segregated recyclate')) 
                        | ( (df['FacilityType'] =='Hazardous landfill') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='In vessel composting') & (df['WasteStreamType'] == 'Food waste')) 
                        | ( (df['FacilityType'] =='In vessel composting') & (df['WasteStreamType'] == 'Green waste')) 
                        | ( (df['FacilityType'] =='In vessel composting') & (df['WasteStreamType'] == 'Mixed green and food waste')) 
                        | ( (df['FacilityType'] =='Incineration with energy recovery') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Incineration with energy recovery') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Incineration without energy recovery') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Inert landfill') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Inert landfill') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Material From WDA') & (df['WasteStreamType'] == 'Comingled recyclate')) 
                        | ( (df['FacilityType'] =='Material From WDA') & (df['WasteStreamType'] == 'Mixed green and food waste')) 
                        | ( (df['FacilityType'] =='Material From WDA') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Material From WDA') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Material From WDA') & (df['WasteStreamType'] == 'Source segregated recyclate')) 
#                         | ( (df['FacilityType'] =='Materials recovery facility') & (df['WasteStreamType'] == 'Comingled recyclate')) 
                        | ( (df['FacilityType'] =='Materials recovery facility') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Mechanical Biological treatment') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Non-hazardous landfill') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Non-hazardous landfill') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Other Method') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='RDF, autoclave, MHT or similar') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Reprocessor - recycling (qu19)') & (df['WasteStreamType'] == 'Green waste')) 
                        | ( (df['FacilityType'] =='Reprocessor - recycling (qu19)') & (df['WasteStreamType'] == 'No Waste Stream Type')) 
                        | ( (df['FacilityType'] =='Reprocessor - recycling (qu19)') & (df['WasteStreamType'] == 'Source segregated recyclate')) 
                        | ( (df['FacilityType'] =='Residual waste MRF') & (df['WasteStreamType'] == 'Residual waste')) 
                        | ( (df['FacilityType'] =='Reuse (qu35)') & (df['WasteStreamType'] == 'Source segregated recyclate')) 
                        | ( (df['FacilityType'] =='Windrow or other composting') & (df['WasteStreamType'] == 'Food waste')) 
                        | ( (df['FacilityType'] =='Windrow or other composting') & (df['WasteStreamType'] == 'Green waste')) 
                        | ( (df['FacilityType'] =='Windrow or other composting') & (df['WasteStreamType'] == 'Mixed green and food waste')) 
                    )][[
                'WasteProcessorOutputId', 'SenderWasteProcessorOutputId',
                'WasteProcessorId', 'WasteStreamId',  
                'Authority', 'AuthorityId', 'WasteManagementAuthority',
                'Quarter',
                "FacilityName", "FacilityName", "NationalFacilityId", "Location_Coordinates", "FacilityAddressGeo",
                "FacilityType", "FacilityType","FacilityTypeId",
                'WasteStreamTypeId', 'WasteStreamType',
                "OutputProcessType", "OutputProcessTypeId",
                'MaterialId', 'Material', 'MaterialGroup', 
                'Tonnes',
                "FacilityIdGenerated", "FacilityIdGenerated", 'FacilityIdGenerated'
                ]]

df_stream_same.columns = ['WasteProcessorOutputId', 'SenderWasteProcessorOutputId',
                'WasteProcessorId', 'WasteStreamId',
                'Authority', 'AuthorityId', 'WasteManagementAuthority',
                'Quarter', 
                "FacilityName_sender", "FacilityName", "NationalFacilityId", "Location_Coordinates", "FacilityAddressGeo",
                "FacilityType_sender", "FacilityType","FacilityTypeId",
                'WasteStreamTypeId', 'WasteStreamType',
                "OutputProcessType", "OutputProcessTypeId",
                'MaterialId', 'Material', 'MaterialGroup',
                'Tonnes',
                "sender_facility","processing_facility", 'FacilityIdGenerated'
                ]


In [12]:
# The rest of edges - subsequent waste streams from Local Authority to Processing Facilities and from Facilities to each other or export

# Pick up sender facility info from another row of dataset, join on 'WasteProcessorOutputId' = 'SenderWasteProcessorOutputId'
# (note that source can be 0 but destination can't - exclude these)  

# Source facility
df_Source_Facility = df[[
                'WasteProcessorOutputId', 'SenderWasteProcessorOutputId',
                'WasteProcessorId', 'WasteStreamId',  
                'AuthorityId',
                'Quarter',
                'WasteStreamTypeId',
                "FacilityType", "FacilityName",
                "FacilityIdGenerated"]]

df_Source_Facility.columns = ['key', 'SenderWasteProcessorOutputId_source',
                'WasteProcessorId', 'WasteStreamId',  
                'AuthorityId',
                'Quarter',
                'WasteStreamTypeId',
                "FacilityType_sender", "FacilityName_sender",
                "sender_facility"]


# Destination Facility 
# Filter out Final Destination which would cause duplicates through key being same for different materials)
# (an alternative would be to join the final destination separately using material id 
# because apparently tonned by material can be diferent in final destination line )                      
df_Destination_Facility = df[~((df['WasteProcessorOutputId']==0)&(df['SenderWasteProcessorOutputId'] ==0) |
                               (df['FacilityTypeId'] == 22)
                              )][[
                'WasteProcessorOutputId', 'SenderWasteProcessorOutputId',
                'WasteProcessorId', 'WasteStreamId',  
                'Authority', 'AuthorityId', 'WasteManagementAuthority',
                'Quarter',
                "FacilityName", "NationalFacilityId", "Location_Coordinates", "FacilityAddressGeo",
                "FacilityType","FacilityTypeId",
                'WasteStreamTypeId', 'WasteStreamType',
                "OutputProcessType", "OutputProcessTypeId",
                'MaterialId', 'Material', 'MaterialGroup', 
                'Tonnes',
                "FacilityIdGenerated", 'FacilityIdGenerated']]

df_Destination_Facility.columns = [
                'WasteProcessorOutputId', 'key',
                'WasteProcessorId', 'WasteStreamId',  
                'Authority', 'AuthorityId', 'WasteManagementAuthority',
                'Quarter',
                "FacilityName", "NationalFacilityId", "Location_Coordinates", "FacilityAddressGeo",
                "FacilityType","FacilityTypeId",
                'WasteStreamTypeId', 'WasteStreamType',
                "OutputProcessType", "OutputProcessTypeId",
                'MaterialId', 'Material', 'MaterialGroup', 
                'Tonnes',
                "processing_facility", 'FacilityIdGenerated']


                 
# Create Edges with both source and destination, excluding rows where material is unknown and final destinations           
df_edges = pd.merge(df_Source_Facility
                     , df_Destination_Facility
                     , how='inner' 
                     , on=['key', 'WasteProcessorId', 'WasteStreamId', 
                          'AuthorityId', 'Quarter','WasteStreamTypeId']
                     )


In [13]:
df_graph = pd.concat([df_stream_same[['Quarter', 'AuthorityId', 'Authority',
       'WasteManagementAuthority', "FacilityName_sender", 'FacilityName',
       'Location_Coordinates', 'FacilityAddressGeo', "FacilityType_sender",'FacilityType', 
       'FacilityTypeId', 'WasteStreamTypeId', 'WasteStreamType', 'OutputProcessType',
       'OutputProcessTypeId', 'MaterialId', 'Material', 'MaterialGroup',
       'Tonnes', 'sender_facility', 'processing_facility', 'FacilityIdGenerated']]
                      , df_edges[[ 'Quarter', 'AuthorityId', 'Authority',
       'WasteManagementAuthority', "FacilityName_sender", 'FacilityName',
       'Location_Coordinates', 'FacilityAddressGeo', "FacilityType_sender", 'FacilityType',  
       'FacilityTypeId', 'WasteStreamTypeId','WasteStreamType', 'OutputProcessType',
       'OutputProcessTypeId', 'MaterialId', 'Material', 'MaterialGroup',
       'Tonnes', 'sender_facility',  'processing_facility', 'FacilityIdGenerated']]]
                     , sort=False)

In [14]:
df_graph['rownum'] = pd.Series(np.arange(len(df_graph))).values

In [15]:
# Create a "material type" called "landfill" becasue it's a kind of final destination
# When facility type is incineration and no material extrated (i.e. burned off), define material as "process loss"
# Don't want to create a system boundary because want to compare tonnes of recovered matrials vs landfilled and incinerated (vanishing) tonnes

df_graph['MaterialGroup'] = df_graph[['MaterialGroup','FacilityType']].apply(lambda row: row[1] if row[1] in ['Inert landfill', 'Non-hazardous landfill', 'Hazardous landfill'] else row[0], axis=1)

df_graph['MaterialGroup'] = df_graph[['MaterialGroup','FacilityType_sender','FacilityType','OutputProcessType']].apply(lambda row: 
                                    row[2] if ((row[1] != row[2]) 
                                               & (row[2] in ['Inert landfill', 'Non-hazardous landfill', 'Hazardous landfill'])
                                               & (row[1] in ['Incineration with energy recovery','RDF, autoclave, MHT or similar','Incineration without energy recovery','Material From WDA', 'Mechanical Biological treatment', 'In vessel composting', 'Incineration with energy recovery', 'Anaerobic or Aerobic Digestion Whole Waste', 'Anaerobic or Aerobic Digestion Segregated']) 
#                                                & (row[0] in ['Unknown','Inert landfill', 'Non-hazardous landfill', 'Hazardous landfill']) 
                                               & (row[3] == 'Contamination (process rejects)')) 
                                    else row[3] if ((row[2] in ['Incineration with energy recovery','RDF, autoclave, MHT or similar','Incineration without energy recovery','Material From WDA', 'Mechanical Biological treatment']) 
                                                    & (row[0] == 'Unknown') 
                                                    & (row[3] in ['Process loss', 'Incinerator bottom ash', 'Incinerator fly ash'])) 
                                    else 'Process loss' if ((row[2] in ['Incineration with energy recovery','RDF, autoclave, MHT or similar','Incineration without energy recovery','Material From WDA', 'Mechanical Biological treatment']) 
                                                            & (row[0] == 'Unknown') 
                                                            & (row[3] == 'Unknown')) 
                                    else row[0], axis=1)

In [ ]:
#df_graph.to_csv('./datasets/waste_graph.csv', encoding="utf8", index=False)